In [ ]:
!wget -c https://www.encodeproject.org/files/ENCFF000MBM/@@download/ENCFF000MBM.fastq.gz -O ENCFF000MBM.fastq.gz
!wget -c https://www.encodeproject.org/files/ENCFF000MBV/@@download/ENCFF000MBV.fastq.gz -O ENCFF000MBV.fastq.gz


In [ ]:
!conda create -y -n rrbs-env -c bioconda fastqc trim-galore cutadapt bowtie2 bismark samtools


In [ ]:
# FastQC (Java tool)
!/home/sagemaker-user/.conda/envs/rrbs-env/opt/fastqc-0.12.1/fastqc --version

# Cutadapt (Python, needed by Trim Galore)
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/cutadapt --version

# Trim Galore (Perl wrapper)
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/trim_galore --version

# Bowtie2 (Aligner)
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bowtie2 --version

# Bismark (Bisulfite aligner)
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bismark --version

# Samtools (for BAM handling)
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools --version


In [ ]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/trim_galore \
  --rrbs \
  --quality 20 \
  --cores 8 \
  --path_to_cutadapt /home/sagemaker-user/.conda/envs/rrbs-env/bin/cutadapt \
  ENCFF000MBM.fastq.gz

!/home/sagemaker-user/.conda/envs/rrbs-env/bin/trim_galore \
  --rrbs \
  --quality 20 \
  --cores 8 \
  --path_to_cutadapt /home/sagemaker-user/.conda/envs/rrbs-env/bin/cutadapt \
  ENCFF000MBV.fastq.gz


In [ ]:
!ls *.txt

In [ ]:
!cat ENCFF000MBM.fastq.gz_trimming_report.txt
!cat ENCFF000MBV.fastq.gz_trimming_report.txt

In [4]:
import os
os.environ["PATH"] = "/home/sagemaker-user/.conda/envs/rrbs-env/bin:" + os.environ["PATH"]


In [ ]:
!mkdir -p genome_hg19
!cp hg19_main.fa genome_hg19/

In [ ]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bismark_genome_preparation --bowtie2 genome_hg19/


In [ ]:
# Sample 1: ENCFF000MBM
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bismark \
  --genome genome_hg19/ \
  -p 32 \
  ENCFF000MBM_trimmed.fq.gz



In [ ]:
# Sample 2: ENCFF000MBV
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bismark \
  --genome genome_hg19/ \
  -p 32 \
  ENCFF000MBV_trimmed.fq.gz
!echo "Finished"

In [ ]:
!ls *.bam

In [11]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools merge -@ 34 merged.bam ENCFF000MBM_trimmed_bismark_bt2.bam ENCFF000MBV_trimmed_bismark_bt2.bam


In [ ]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools sort -@ 34 -o merged.sorted.bam merged.bam

In [14]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools index merged.sorted.bam


In [ ]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/bismark_methylation_extractor --samtools_path /home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools --bedGraph --gzip --multicore 34 --buffer_size 2G --genome_folder genome_hg19 merged.sorted.bam


In [ ]:
!ls -lh *.gz

In [18]:
!gunzip merged.sorted.bedGraph.gz


In [19]:
!sort -k1,1 -k2,2n merged.sorted.bedGraph > merged.sorted.sorted.bedGraph


In [ ]:
!ls genome_hg19

In [24]:
!/home/sagemaker-user/.conda/envs/rrbs-env/bin/samtools faidx genome_hg19/hg19_main.fa

In [ ]:
!cut -f1,2 genome_hg19/hg19_main.fa.fai > genome_hg19/hg19.chrom.sizes


In [ ]:
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedGraphToBigWig
!chmod +x bedGraphToBigWig
!sudo mv bedGraphToBigWig /usr/local/bin/


In [ ]:

# 4. Convert to BigWig
!bedGraphToBigWig merged.sorted.sorted.bedGraph genome_hg19/hg19.chrom.sizes merged.sorted.bw

In [ ]:
!head merged.sorted.sorted.bedGraph

In [ ]:
!awk 'NF<3' merged.sorted.sorted.bedGraph


In [3]:
!awk 'NF>=3' merged.sorted.sorted.bedGraph > fixed.bedGraph


In [ ]:
!bedGraphToBigWig fixed.bedGraph genome_hg19/hg19.chrom.sizes merged.sorted.bw
